In [11]:
import warnings
warnings.filterwarnings('ignore')
#
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
# from backtesting._plotting import set_bokeh_output
# set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

In [12]:
from utilities import *

In [13]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [14]:
df = dataset.copy()

In [15]:
df = df[df.index > '2022-11-01 00:00:00']

In [16]:
df = add_pivots(df, window=3)

In [17]:
df

,Open,High,Low,Close,Volume,pivoth,pivotl
Date,,,,,,,
2022-11-01 09:00:00,1030.5,1036.0,1028.3,1035.0,15042,False,False
2022-11-01 09:05:00,1035.1,1038.2,1034.4,1035.6,11554,False,False
2022-11-01 09:10:00,1035.3,1039.0,1035.3,1037.3,7898,False,False
2022-11-01 09:15:00,1037.0,1040.9,1036.9,1040.9,9896,False,False
2022-11-01 09:20:00,1040.9,1042.6,1039.1,1041.1,9138,True,False
...,...,...,...,...,...,...,...
2025-10-21 14:10:00,1927.5,1933.0,1918.7,1922.0,16658,True,False
2025-10-21 14:15:00,1922.0,1927.5,1919.2,1921.5,8361,False,False
2025-10-21 14:20:00,1921.5,1926.0,1902.9,1906.7,16009,False,False


In [18]:
plot_candles_with_pivots(df, 0, 200)

In [19]:
LOOKBACK = 40 # 40
PIVOT_WINDOW = 3 # 3
MAX_HIGH_SLOPE = 0.0002 # 0.0001
MIN_LOW_SLOPE = 0.0005 # 0.0005
MIN_R2 = 0.7 # 0.7

In [20]:
signal_len = 4        # breakout bar + 2 more

df["triangle_flag"] = False #np.nan

for i in range(30, len(df)):
    flag = detect_flag(df, i, 
                       lookback= LOOKBACK, 
                       pivot_window= PIVOT_WINDOW, 
                       max_high_slope= MAX_HIGH_SLOPE, 
                       min_low_slope = MIN_LOW_SLOPE,
                       min_r2= MIN_R2)
    if flag:
        dt = df.index[i]
        print(f"Flag confirmed with breakout bar {i}, {dt}")
        print(flag)
        end_pos = min(i + signal_len, len(df))
        df.iloc[i:end_pos, df.columns.get_loc("triangle_flag")] = True

Flag confirmed with breakout bar 5274, 2023-04-03 10:55:00
{'highs': [(5260, 1081.7), (5261, 1081.7), (5268, 1081.2), (5269, 1081.2)], 'lows': [(5240, 1068.0), (5250, 1066.0), (5264, 1079.0), (5271, 1079.6)], 'slope_high': np.float64(-0.0001849368902861898), 'slope_low': np.float64(0.004454176955691163)}
Flag confirmed with breakout bar 5275, 2023-04-03 11:00:00
{'highs': [(5260, 1081.7), (5261, 1081.7), (5268, 1081.2), (5269, 1081.2)], 'lows': [(5240, 1068.0), (5250, 1066.0), (5264, 1079.0), (5271, 1079.6)], 'slope_high': np.float64(-0.0001849368902861898), 'slope_low': np.float64(0.004454176955691163)}
Flag confirmed with breakout bar 5276, 2023-04-03 11:05:00
{'highs': [(5260, 1081.7), (5261, 1081.7), (5268, 1081.2), (5269, 1081.2)], 'lows': [(5240, 1068.0), (5250, 1066.0), (5264, 1079.0), (5271, 1079.6)], 'slope_high': np.float64(-0.0001849368902861898), 'slope_low': np.float64(0.004454176955691163)}
Flag confirmed with breakout bar 5277, 2023-04-03 11:10:00
{'highs': [(5260, 1081.

In [21]:
df

,Open,High,Low,Close,Volume,pivoth,pivotl,triangle_flag
Date,,,,,,,,
2022-11-01 09:00:00,1030.5,1036.0,1028.3,1035.0,15042,False,False,False
2022-11-01 09:05:00,1035.1,1038.2,1034.4,1035.6,11554,False,False,False
2022-11-01 09:10:00,1035.3,1039.0,1035.3,1037.3,7898,False,False,False
2022-11-01 09:15:00,1037.0,1040.9,1036.9,1040.9,9896,False,False,False
2022-11-01 09:20:00,1040.9,1042.6,1039.1,1041.1,9138,True,False,False
...,...,...,...,...,...,...,...,...
2025-10-21 14:10:00,1927.5,1933.0,1918.7,1922.0,16658,True,False,False
2025-10-21 14:15:00,1922.0,1927.5,1919.2,1921.5,8361,False,False,False
2025-10-21 14:20:00,1921.5,1926.0,1902.9,1906.7,16009,False,False,False


In [22]:
mask = df["triangle_flag"]
label_index = df.index[mask]
int_index   = np.flatnonzero(mask.values)

In [23]:
int_index

array([ 5274,  5275,  5276,  5277,  5278,  5279,  5280,  5281,  7832,
        7833,  7834,  7835,  7836,  7837,  7838, 10377, 10378, 10379,
       10380, 10381, 10382, 10383, 10384, 10385, 10386, 10387, 10388,
       11849, 11850, 11851, 11852, 11853, 11854, 11855, 18391, 18392,
       18393, 18394, 18395, 18396, 18399, 18400, 18401, 18402, 18403,
       18404, 21165, 21166, 21167, 21168, 21169, 21170, 21171, 21172,
       23778, 23779, 23780, 23781, 23782, 23783, 23784, 23785, 23786,
       23855, 23856, 23857, 23858, 23859, 23860, 23861, 23862, 23863,
       26745, 26746, 26747, 26748, 26749, 26750, 26751, 26752, 26753,
       26754, 26755, 26756, 28412, 28413, 28414, 28415, 28416, 28417,
       28418, 28419, 28420, 28421, 28422, 28423, 28424, 28425])

In [24]:
from backtesting import Backtest, Strategy

df['my_signal'] = df['triangle_flag']

class ShortSignalStrategy(Strategy):
    """
    Short when df['my_signal'] is True.
    Stop-loss  = High + sl_buffer * High
    Take-profit = entry – rr * (stop−entry)
    """
    sl_buffer = 0.01     # 1 % above high
    rr        = 3      # risk-reward multiple

    def init(self):
        # cache series for speed
        self.signal = self.data.my_signal
        self.high   = self.data.High
        self.close  = self.data.Close

    def next(self):
        i = len(self.data) - 1

        # open short
        if self.signal[i] and not self.position:
            entry   = self.close[i]
            stop    = self.high[i] * (1 + self.sl_buffer)
            tp      = entry - self.rr * (stop - entry)
            self.sell(size=0.99, sl=stop, tp=tp)

bt = Backtest(
    df,
    ShortSignalStrategy,
    cash=10000,
    commission=0.0005,
    trade_on_close=True, # enter at close of signal bar
    exclusive_orders=True,
    margin=1/10
)

stats = bt.run()
stats

Start                     2022-11-01 09:00:00
End                       2025-10-21 14:45:00
Duration                   1085 days 05:45:00
Exposure Time [%]                    10.27076
Equity Final [$]                   4162.51594
Equity Peak [$]                   15209.24236
Commissions [$]                     840.77206
Return [%]                          -58.37484
Buy & Hold Return [%]                 83.5942
Return (Ann.) [%]                     -25.775
Volatility (Ann.) [%]                28.82422
CAGR [%]                            -18.41471
Sharpe Ratio                         -0.89421
Sortino Ratio                        -0.88829
Calmar Ratio                         -0.35487
Alpha [%]                           -32.61053
Beta                                 -0.30821
Max. Drawdown [%]                   -72.63167
Avg. Drawdown [%]                     -4.2922
Max. Drawdown Duration      854 days 01:15:00
Avg. Drawdown Duration       25 days 18:30:00
# Trades                          

In [25]:
# optimise the RR multiple if you wish
stats_opt = bt.optimize(rr=list(np.arange(1.5, 3.1, 0.5)))
stats_opt['_strategy']

<Strategy ShortSignalStrategy(rr=2.5)>

In [ ]:
# Precompute the flags (stores lists of (index, price) in the breakout bar row)
df = precompute_triangle_flags(
    df,
    lookback=LOOKBACK, pivot_window=PIVOT_WINDOW,
    max_high_slope=MAX_HIGH_SLOPE, min_low_slope=MIN_LOW_SLOPE, min_r2=MIN_R2,
    signal_len=3,
)

In [ ]:
# Plot a region
plot_candles_with_pivots_and_flags(df, start_idx=3800, end_idx=4200)